# <center><font color="red"> https://bit.ly/ptpjb-2021-18</font>
# <center><font color="blue">18 - Introduction to Streamlit for Machine Learning Deployment</font>

<center><img alt="" src="images/cover_ptpjb_2021.png"/></center> 

## <center><font color="blue">tau-data Indonesia</font><br>(C) Taufik Sutanto - 2021</center>
<center><a href="https://tau-data.id">https://tau-data.id</a> ~ <a href="mailto:taufik@tau-data.id">taufik@tau-data.id</a></center>

# 18 - Introduction to Streamlit for Machine Learning Deployment

<img src="images/streamlit.jpg"/>

link: https://streamlit.io/

<br>
<div style="text-align: center">
    <video controls src="videos/Streamlit.mp4" width="700" >
<div/>

## Deploy Anomaly Detection App

kita akan coba bagaimana melakukan deployment sederhana menggunakan streamlit pada model anomaly detection yang telah kita buat. hasil deployment yang akan kita buat akan memiliki tampilan seperti berikut:

<img src="images/streamlit app.jpg" width="700"/>

## Persiapan
**Lakukan instalasi streamlit pada command prompt (cmd):**
- `> pip install streamlit`
- `> streamlit hello`

**Menyiapkan file yang diperlukan:**
- model anomaly detection yang telah dibuat
- feature scaling yang digunakan
- data untuk mencoba aplikasi anomaly detection

file-file di atas dapat dipersiapkan pada modul jupyter notebook **14 - Deep Learning ~ LSTM** dan di simpan pada folder deployment.

**Membuat text-file python**
kemudian pada folder deployment tersebut kita buat text-file dengan format `.py` untuk menuliskan code deployment menggunakan streamlit.

<img src="images/text-file 1.jpg"/>
<img src="images/text-file 2.jpg"/>

## Code in Python

Pada file `LSTM_AD.py` kita tuliskan code seperti di bawah ini. kemudian kita akan bahas kegunaannya baris-perbaris.

In [ ]:
"""The App."""

import pandas as pd
import numpy as np
import streamlit as st
import keras
import pickle
import plotly.graph_objects as go

# Create sliding window
def sliding_window(seq, window_size):
    sub_seq, next_values = [], []
    for i in range(len(seq)-window_size):
        sub_seq.append(seq[i:i+window_size])
        next_values.append([seq[i+window_size]])
    X = np.array(sub_seq)
    y = np.array(next_values)
    return X,y

window_size = 10

# Load the model from the file
model = keras.models.load_model('anomaly_detection')

# load the scaler
scaler = pickle.load(open('scaler.pkl', 'rb'))

threshold = 18.97461056150496

st.write("""
# LSTM Anomaly Detection App for Web Traffic Data
""")

st.write("""
### Data format and must be greater than 10 timestamps
| timestamp  | value   |
| -----------|:-------:|
| 1          | 10      |
| 2          | 7       |
| 3          | 17      |
""")

uploaded_file = st.file_uploader("Choose a file", type='csv')

if uploaded_file is not None:

    df = pd.read_csv(uploaded_file)
    df['scaled'] = scaler.transform(df[['value']])
    
    X, y = sliding_window(df['scaled'], window_size)
    
    # Reshape input to be [samples, time steps, features]
    X = X.reshape(X.shape[0], 1, X.shape[1])
    
    predict = scaler.inverse_transform(model.predict(X))
    y = scaler.inverse_transform(y)
    
    abs_error = np.abs(y - predict)

    test_score_df = pd.DataFrame()
    test_score_df['timestamp'] = df['timestamp'][window_size:]
    test_score_df['value'] = df['value'][window_size:]
    test_score_df['loss'] = abs_error
    test_score_df['threshold'] = threshold
    test_score_df['anomaly_hat'] = 0
    test_score_df.loc[test_score_df.loss >= test_score_df.threshold, 'anomaly_hat'] = 1
    
    anomalies = test_score_df.loc[test_score_df['anomaly_hat'] == 1]

    st.write("Visualize Detected Anomalies from Data")  

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=test_score_df['timestamp'], y=test_score_df['value'], name='value'))
    fig.add_trace(go.Scatter(x=anomalies['timestamp'], y=anomalies['value'], mode='markers', name='Anomaly'))
    fig.update_layout(showlegend=True, title='Detected anomalies')
    st.plotly_chart(fig)

**Import modul/library yang diperlukan**

In [ ]:
import pandas as pd
import numpy as np
import streamlit as st
import keras
import pickle
import plotly.graph_objects as go

**Membuat fungsi sliding window serta menentukan window size sesuai dengan model anomaly detection yang digunakan**

In [ ]:
# Create sliding window
def sliding_window(seq, window_size):
    sub_seq, next_values = [], []
    for i in range(len(seq)-window_size):
        sub_seq.append(seq[i:i+window_size])
        next_values.append([seq[i+window_size]])
    X = np.array(sub_seq)
    y = np.array(next_values)
    return X,y

window_size = 10

**Memuat model anomaly detection dan feature scaling serta menentukan threshold dari hasil model anomaly detection yang dibuat**

In [ ]:
# Load the model from the file
model = keras.models.load_model('anomaly_detection')

# load the scaler
scaler = pickle.load(open('scaler.pkl', 'rb'))

threshold = 18.209396362304688

**Menampilkan header dan contoh format data yang benar**

In [ ]:
st.write("""
# LSTM Anomaly Detection App for Web Traffic Data
""")

st.write("""
### Data format and must be greater than 10 timestamps
| timestamp  | value   |
| -----------|:-------:|
| 1          | 10      |
| 2          | 7       |
| 3          | 17      |
""")

Fungsi st.write() dari modul streamlit berfungsi untuk menampilkan sesuatu dalam format `html`. Agar dapat menulis bahasa `html` dengan beberapa baris maka kita gunakan tanda kutip tiga `(""" <html> """)`

Dari code di atas kita akan peroleh tampilan sebagai berikut:

<img src="images/header.jpg"/>

**Membuat tombol upload**

In [ ]:
uploaded_file = st.file_uploader("Choose a file", type='csv')

Fungsi st.file_uploader() dari modul streamlit berfungsi untuk membuat tombol upload. Argumen `"Choose a file"` dan `type='csv'` berarti kita menentukan judul pada tombol upload dan tipe file yang dapat diupload.

Dari code di atas kita akan peroleh tampilan sebagai berikut:

<img src="images/file upload.jpg"/>

**Memproses data yang diupload**

Setelah data diupload kita gunakan code `if uploaded_file is not None:` untuk memperoses data tersebut dengan langkah-langkah seperti pada modul jupyter notebook **14 - Deep Learning ~ LSTM**

untuk menampilkan grafik kita gunakan st.plotly_chart() dari modul streamlit dengan argumen `fig` yaitu variabel grafik dari modul plotly

In [ ]:
if uploaded_file is not None:

    df = pd.read_csv(uploaded_file)
    df['scaled'] = scaler.transform(df[['value']])
    
    X, y = sliding_window(df['scaled'], window_size)
    
    # Reshape input to be [samples, time steps, features]
    X = X.reshape(X.shape[0], 1, X.shape[1])
    
    predict = scaler.inverse_transform(model.predict(X))
    y = scaler.inverse_transform(y)
    
    abs_error = np.abs(y - predict)

    test_score_df = pd.DataFrame()
    test_score_df['timestamp'] = df['timestamp'][window_size:]
    test_score_df['value'] = df['value'][window_size:]
    test_score_df['loss'] = abs_error
    test_score_df['threshold'] = threshold
    test_score_df['anomaly_hat'] = 0
    test_score_df.loc[test_score_df.loss >= test_score_df.threshold, 'anomaly_hat'] = 1
    
    anomalies = test_score_df.loc[test_score_df['anomaly_hat'] == 1]

    st.write("Visualize Detected Anomalies from Data")  

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=test_score_df['timestamp'], y=test_score_df['value'], name='value'))
    fig.add_trace(go.Scatter(x=anomalies['timestamp'], y=anomalies['value'], mode='markers', name='Anomaly'))
    fig.update_layout(showlegend=True, title='Detected anomalies')
    st.plotly_chart(fig)

Dari code di atas kita akan peroleh tampilan sebagai berikut setelah file diupload:<br>
<img src="images/uploaded file.jpg"/>

## Penutup
Masih banyak lagi cara untuk deployment model machine learning menggunakan streamlit. kita dapat mengeksplor lagi dokumentasi streamlit pada link berikut: https://docs.streamlit.io/en/stable/api.html ataupun melihat hasil deployment yang telah dibagikan pada link berikut: https://streamlit.io/gallery. Selain itu dapat juga didesain terlebih dahulu deployment yang ingin dilakukan agar lebih terarah.

# <center><font color="blue">Akhir Modul 18 - Introduction to Streamlit for Machine Learning Deployment
    
<hr />
<img alt="" src="images/meme-cartoon/meme deployment.jpg" />